In [18]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import json 
# Lead chunks.json into a dictionary
with open('chunks.json', 'r') as f:
    chunks = json.load(f)

In [ ]:
#Convert dict to a tuple of (game, review)
reviews = []
for game in chunks:
    for review in chunks[game]:
        reviews.append((game, review))


In [ ]:
# Create a list for reviews and a list for games
review_list = []
game_list = []
for review in reviews:
    game_list.append(review[0])
    review_list.append(review[1])


In [ ]:
# Convert each game to a dictionary of gameDict["game"] = [game]
gameDictList = []
for game in game_list:
    gameDict = {}
    gameDict["game"] = [game]
    gameDictList.append(gameDict)
gameDictList

In [19]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [20]:
import os
import dotenv

dotenv.load_dotenv('env')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

In [21]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [22]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain" # put in the name of your pinecone index here

In [23]:
#docsearch = Pinecone.from_texts(review_list, embeddings, metadatas=gameDictList, index_name=index_name) #Initial population !!
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [24]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [25]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [39]:
query = """" I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic."""
docs = docsearch.similarity_search_with_score(query, k=5)


In [63]:
games = []
for i in range(len(docs)):
    games.append(docs[i][0].metadata["game"][0])
games

['House Builder', '7 Days to Die', 'Terraria', 'Terraria', "Don't Starve"]

In [64]:
game_name = games[0]

In [53]:
query = """" I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic."""
game_docs = docsearch.similarity_search(query, k=5, filter={"game": game_name})

In [54]:
game_docs

[Document(page_content="Minecraft was too boring for me (I'm not a creator for creations sake). Terraria was fun, I enjoyed the additional combat focus.  DayZ didn't have 'it' for me, I didn't like the lack of building (at least when I played it) Ark was fun (dinos!), but it got stale as there was no forced conflict. No reason to explore, no reason to keep building. So it fell into the minecraft trap.  7 Days to Die allows me to be creative, but gives me a good solid foundation of conflict and purpose to my building. Not only that, but it creatively forces/encourages me to get out of my safe keep and risk exploring which causes me to engage with the game content in a very enjoyable fashion.  This was a recommendation from a friend based on the steam sale, and I've dedicated 90% of my gaming hours to this since I bought it over the holiday break. Strongly recommend!im 65 and been playing games since the early 80s, i have around 1500 hrs gameplay in 7 days to die game and must say that t

In [55]:
query = "Reccommend this game to me, If you wouldn't reccommend it always start your answer wtih a No. Make relations to my query: " + query
chain.run(input_documents=game_docs, question=query)

" Yes, 7 Days to Die is a great game for what you're looking for. It has pixelated graphics, single-player and multiplayer modes, and allows you to explore and modify an open world. You can dig into the earth, gather resources, and build structures. There is a progression system tied to combating various bosses and enemies. The game also includes crafting mechanics, allowing you to utilize the resources you gather to create new items, tools, and weapons. Additionally, there are different biomes and events to keep the gameplay engaging and dynamic."

In [67]:
for i in range (len(game_docs)):
    print(i)

0
1
2
3
4
